In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

In [3]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 64
display_step = 20

In [4]:
# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.8 # Dropout, probability to keep units

In [5]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [6]:
# Create custom model
def conv2d(name, l_input, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(l_input, w, strides=[1, 1, 1, 1], padding='SAME'),b), name=name)

def max_pool(name, l_input, k):
    return tf.nn.max_pool(l_input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME', name=name)

def norm(name, l_input, lsize=4):
    return tf.nn.lrn(l_input, lsize, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name=name)

def customnet(_X, _weights, _biases, _dropout):
    # Reshape input picture
    _X = tf.reshape(_X, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d('conv1', _X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    pool1 = max_pool('pool1', conv1, k=2)
    # Apply Normalization
    norm1 = norm('norm1', pool1, lsize=4)
    # Apply Dropout
    norm1 = tf.nn.dropout(norm1, _dropout)

    # Convolution Layer
    conv2 = conv2d('conv2', norm1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    pool2 = max_pool('pool2', conv2, k=2)
    # Apply Normalization
    norm2 = norm('norm2', pool2, lsize=4)
    # Apply Dropout
    norm2 = tf.nn.dropout(norm2, _dropout)

    # Convolution Layer
    conv3 = conv2d('conv3', norm2, _weights['wc3'], _biases['bc3'])
    # Max Pooling (down-sampling)
    pool3 = max_pool('pool3', conv3, k=2)
    # Apply Normalization
    norm3 = norm('norm3', pool3, lsize=4)
    # Apply Dropout
    norm3 = tf.nn.dropout(norm3, _dropout)
    #conv4
    conv4 = conv2d('conv4', norm3, _weights['wc4'], _biases['bc4'])
    # Max Pooling (down-sampling)
    pool4 = max_pool('pool4', conv4, k=2)
    # Apply Normalization
    norm4 = norm('norm4', pool4, lsize=4)
    # Apply Dropout
    norm4 = tf.nn.dropout(norm4, _dropout)
    # Fully connected layer
    dense1 = tf.reshape(norm4, [-1, _weights['wd1'].get_shape().as_list()[0]]) # Reshape conv3 output to fit dense layer input
    dense1 = tf.nn.relu(tf.matmul(dense1, _weights['wd1']) + _biases['bd1'], name='fc1') # Relu activation

    dense2 = tf.nn.relu(tf.matmul(dense1, _weights['wd2']) + _biases['bd2'], name='fc2') # Relu activation

    # Output, class prediction
    out = tf.matmul(dense2, _weights['out']) + _biases['out']
    return out

In [7]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64])),
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'wc3': tf.Variable(tf.random_normal([3, 3, 128, 256])),
    'wc4': tf.Variable(tf.random_normal([2, 2, 256, 512])),
    'wd1': tf.Variable(tf.random_normal([2*2*512, 1024])), 
    'wd2': tf.Variable(tf.random_normal([1024, 1024])),
    'out': tf.Variable(tf.random_normal([1024, 10]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([64])),
    'bc2': tf.Variable(tf.random_normal([128])),
    'bc3': tf.Variable(tf.random_normal([256])),
    'bc4': tf.Variable(tf.random_normal([512])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'bd2': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [8]:
# Construct model
pred = customnet(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [14]:
# 可视化
tf.summary.scalar("loss", cost)
tf.summary.scalar("accuracy", accuracy)
# Merge all summaries to a single operator
merged_summary_op = tf.summary.merge_all()

In [16]:
with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.summary.FileWriter('./logs', graph_def=sess.graph_def)
    step = 1
    while step*batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: batch_xs, y:batch_ys, keep_prob: dropout})
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.})
            loss = sess.run(cost, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            summary_writer.add_summary(summary_str, step)
        step += 1
    print("Optimization Finished!")
    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))

Iter 1280, Minibatch Loss= 86461.703125, Training Accuracy= 0.23438
Iter 2560, Minibatch Loss= 80945.945312, Training Accuracy= 0.32812
Iter 3840, Minibatch Loss= 71981.726562, Training Accuracy= 0.26562
Iter 5120, Minibatch Loss= 56036.445312, Training Accuracy= 0.40625
Iter 6400, Minibatch Loss= 46967.976562, Training Accuracy= 0.31250
Iter 7680, Minibatch Loss= 65222.226562, Training Accuracy= 0.29688
Iter 8960, Minibatch Loss= 76156.218750, Training Accuracy= 0.21875
Iter 10240, Minibatch Loss= 27992.527344, Training Accuracy= 0.46875
Iter 11520, Minibatch Loss= 33608.617188, Training Accuracy= 0.35938
Iter 12800, Minibatch Loss= 39848.328125, Training Accuracy= 0.40625
Iter 14080, Minibatch Loss= 34595.750000, Training Accuracy= 0.31250
Iter 15360, Minibatch Loss= 27668.664062, Training Accuracy= 0.40625
Iter 16640, Minibatch Loss= 28438.660156, Training Accuracy= 0.42188
Iter 17920, Minibatch Loss= 18373.679688, Training Accuracy= 0.46875
Iter 19200, Minibatch Loss= 19141.173828,

Iter 152320, Minibatch Loss= 2569.232422, Training Accuracy= 0.75000
Iter 153600, Minibatch Loss= 1164.939697, Training Accuracy= 0.85938
Iter 154880, Minibatch Loss= 988.809814, Training Accuracy= 0.81250
Iter 156160, Minibatch Loss= 810.059570, Training Accuracy= 0.79688
Iter 157440, Minibatch Loss= 1880.863281, Training Accuracy= 0.71875
Iter 158720, Minibatch Loss= 1756.005127, Training Accuracy= 0.71875
Iter 160000, Minibatch Loss= 2010.363281, Training Accuracy= 0.73438
Iter 161280, Minibatch Loss= 1815.369141, Training Accuracy= 0.75000
Iter 162560, Minibatch Loss= 909.377930, Training Accuracy= 0.84375
Iter 163840, Minibatch Loss= 874.129578, Training Accuracy= 0.85938
Iter 165120, Minibatch Loss= 2770.332031, Training Accuracy= 0.73438
Iter 166400, Minibatch Loss= 1247.842285, Training Accuracy= 0.76562
Iter 167680, Minibatch Loss= 1409.227295, Training Accuracy= 0.79688
Iter 168960, Minibatch Loss= 1445.460449, Training Accuracy= 0.79688
Iter 170240, Minibatch Loss= 1054.5507